Here we want to investigate if the limiting of NN matches via the `global-phase` ranking is viable.
Main questions are:
- does the re-scoring trick work at all? Yes, with an exception to passing RRF score to a function.
- What happens with items that dont have the NN score at all? They get 0 score, so it must be worked out when adding a condition.
- How many items can be reranked this way (content nodes)? Depends on the latency budget
- what is the cheapest way to get features for global phase? TO BE DONE

Start Vespa Docker container:
```shell
docker run \
  --detach --rm \
  --name vespa-global-phase \
  --hostname vespa-global-phase \
  --publish 0.0.0.0:8080:8080 \
  --publish 0.0.0.0:19050:19050 \
  --publish 0.0.0.0:19071:19071 \
  vespaengine/vespa:8.484.1
```

In [1]:
# Start Vespa with all the required functionality present
!docker run --detach --rm --name vespa-global-phase --hostname vespa-global-phase --publish 0.0.0.0:8080:8080 --publish 0.0.0.0:19050:19050 --publish 0.0.0.0:19071:19071 vespaengine/vespa:8.484.1

docker: Error response from daemon: Conflict. The container name "/vespa-global-phase" is already in use by container "ed7ed5314f6946c7af468aa48c24bd245ffe1e1363540a1b927f3f7d5796f036". You have to remove (or rename) that container to be able to reuse that name.
See 'docker run --help'.


In [19]:
from vespa.package import (ApplicationPackage, Field, Schema, Document, RankProfile, HNSW, GlobalPhaseRanking, Function)
from vespa.deployment import VespaDocker
from vespa.io import VespaResponse

vap = ApplicationPackage(
    name="anncornercase",
    schema=[
        Schema(
            name="ann",
            document=Document(
                fields=[
                    Field(
                        name="filter",
                        type="int",
                        indexing=["attribute", "summary"],
                        attribute=["fast-search"],
                    ),
                    Field(
                        name="embedding",
                        type="tensor<float>(d0[1])",
                        indexing=["attribute", "index"],
                        ann=HNSW(
                            distance_metric="euclidean",
                            max_links_per_node=16,
                            neighbors_to_explore_at_insert=200,
                        ))
                ]
            ),
            rank_profiles=[
                RankProfile(
                    name="ann",
                    inputs=[
                        ("query(q)", "tensor<float>(d0[1])"),
                        ("query(ann_hits)", "double", "1"),
                    ],
                    functions=[
                        Function(
                            name='nearest_neighbor_closeness',
                            expression='rawScore(embedding)',
                        ),
                    ],
                    first_phase="nearest_neighbor_closeness"
                ),
                RankProfile(
                    inherits='ann',
                    name="ann_global_cut",
                    functions=[
                        #It seems that a function in the global phase can't take in reciprocal_rank output
                        # Function(
                        #     name="function_on_reciprocal_rank",
                        #     args=["rr_score"],
                        #     expression="if(rr_score >= 1/2.0, 1.0, 0.0)"
                        # ),
                        Function(
                            name='rrf_inside_function',
                            args=[],
                            expression='if(reciprocal_rank(nearest_neighbor_closeness, 0) >= 1.0/1.0, 1.0, 0.0)',
                        ),
                    ],
                    first_phase="nearest_neighbor_closeness + 1",
                    global_phase=GlobalPhaseRanking(
                        rerank_count=2,
                        # expression="firstPhase * function_on_reciprocal_rank(reciprocal_rank(nearest_neighbor_distance, 0))"
                        expression="firstPhase * if(reciprocal_rank(nearest_neighbor_closeness, 0) >= 1.0/query(ann_hits) || nearest_neighbor_closeness == 0, firstPhase, 0.0)",

                        # expression="firstPhase * rrf_inside_function()",
                        # TODO: I'll have to implement it, to have a complete control.
                        rank_score_drop_limit=0.0,
                    ),
                    match_features=['nearest_neighbor_closeness', 'firstPhase'],
                )
            ]
        )
    ]
)

In [20]:
vap_file_name = "global-phase-trick.zip"
vap.to_zipfile(vap_file_name)
# Deploy VAP zip file
! vespa deploy global-phase-trick.zip -t http://localhost:19071

Uploading application package... done;1m⢿
Success: Deployed 'global-phase-trick.zip' with session ID 2


In [7]:
from vespa.application import Vespa
prod_vespa_host = "http://localhost"
app = Vespa(url=prod_vespa_host, port=8080)

In [14]:
# Create and feed 100 dummy docs
docs = [
    {
        'id': f'{i}',
        'fields': {
            'filter': i,
            'embedding': [i]
        }
    } for i in range(100)]
def callback(response: VespaResponse, document_id: str):
    if not response.is_successful():
        print(f"Error when feeding document {document_id}: {response.get_json()}")

app.feed_iterable(docs, schema="ann", namespace="ann", callback=callback)

In [16]:
resp = app.query(body={
    'yql': """
        select *
        from ann
        where (filter = 99 OR (({targetHits:1, approximate: true}nearestNeighbor(embedding, q)) AND filter < 50))
        """,
    'hits': 10,
    # 'ranking': 'ann',
    "input.query(q)": [10],
    "input.query(ann_hits)": 1,
    'ranking.globalPhase.rerankCount': 100,
    'ranking.globalPhase.rankScoreDropLimit': 1.0,
    "ranking.matching.approximateThreshold": 0.99,
    'ranking.profile': 'ann_global_cut',
    'presentation.summary': 'default',
    'trace.level': 2,
})
resp.json

{'trace': {'children': [{'message': "Using query profile 'default' of type 'root'"},
   {'message': "Invoking chain 'vespa' [com.yahoo.prelude.statistics.StatisticsSearcher@native -> com.yahoo.prelude.querytransform.PhrasingSearcher@vespa -> ... -> federation@native]"},
   {'children': [{'message': 'YQL query parsed: [select * from ann where (filter = 99 OR ({targetNumHits: 1}nearestNeighbor(embedding, q) AND filter < 50))]'},
     {'message': 'Federating to [anncornercase_content]'},
     {'children': [{'message': 'Stemming: [select * from ann where (filter = 99 OR ({targetNumHits: 1}nearestNeighbor(embedding, q) AND filter < 50)) timeout 493]'},
       {'message': 'Lowercasing: [select * from ann where (filter = 99 OR ({targetNumHits: 1}nearestNeighbor(embedding, q) AND filter < 50)) timeout 493]'},
       {'message': 'anncornercase_content.num0 search to dispatch: query=[OR filter:99 (AND NEAREST_NEIGHBOR {field=embedding,queryTensorName=q,hnsw.exploreAdditionalHits=0,distanceThresh

The ranking above explained:
- the first hit has relevance = 1 because its `rawScore = 1` and during the global phase reranking the score was not modified as it matches the positive positional adjustment condition.
- the second hit has relevance = 0, because in global-phase reranking it didn't match the positive positional adjustment condition.
- The third hit has relevance = 0, and it is a good question why exactly.
- The fourth hit `relevance=-0.25` (yes, negative), I guess it is because the scores are scaled to be strictly lower than those after re-scoring.
- The 5th hit `relevance=-0.5` because scaled, and t firstPhase was 0.

In [52]:
# the `rawScore` is equal to the `closeness` which for HNSW with distance `prenormalized-angular` is calculated as:
# rawScore = 1 / (1 + sqrt((q - d)^2))

In [11]:
# Exact nearest neighbor search (i.e. with `approximate: false`) takes into account the target hits in some interesting way.
resp = app.query(body={
    'yql': """
        select *
        from ann
        where (filter = 99 OR (({targetHits:1, approximate: false}nearestNeighbor(embedding, q)) AND filter < 50))
        """,
    'hits': 10,
    # 'ranking': 'ann',
    "input.query(q)": [10],
    "ranking.matching.approximateThreshold": 0.99,
    'ranking.profile': 'ann_global_cut',
    'presentation.summary': 'default',
})
print(f'Hit count with targetHits=1 is {resp.json['root']['fields']['totalCount']}')

resp = app.query(body={
    'yql': """
        select *
        from ann
        where (filter = 99 OR (({targetHits:10, approximate: false}nearestNeighbor(embedding, q)) AND filter < 50))
        """,
    'hits': 10,
    # 'ranking': 'ann',
    "input.query(q)": [10],
    "ranking.matching.approximateThreshold": 0.99,
    'ranking.profile': 'ann_global_cut',
    'presentation.summary': 'default',
})
print(f'Hit count with targetHits=10 is {resp.json['root']['fields']['totalCount']}')

resp = app.query(body={
    'yql': """
        select *
        from ann
        where (filter = 99 OR (({targetHits:50, approximate: false}nearestNeighbor(embedding, q)) AND filter < 50))
        """,
    'hits': 10,
    # 'ranking': 'ann',
    "input.query(q)": [10],
    "ranking.matching.approximateThreshold": 0.99,
    'ranking.profile': 'ann_global_cut',
    'presentation.summary': 'default',
})
print(f'Hit count with targetHits=50 is {resp.json['root']['fields']['totalCount']}')

Hit count with targetHits=1 is 0
Hit count with targetHits=10 is 0
Hit count with targetHits=50 is 0


In [107]:
# it somewhat matches the `approximate: true` where hit estimate falls under the `ranking.matching.approximateThreshold` value.
resp = app.query(body={
    'yql': """
        select *
        from ann
        where (filter = 99 OR (({targetHits:10, approximate: true}nearestNeighbor(embedding, q)) AND filter < 50))
        """,
    'hits': 10,
    # 'ranking': 'ann',
    "input.query(q)": [10],
    "ranking.matching.approximateThreshold": 0.99,
    'ranking.profile': 'ann_global_cut',
    'presentation.summary': 'default',
})
print(f'Hit count with targetHits=10 is {resp.json['root']['fields']['totalCount']}')

Hit count with targetHits=10 is 17


In [108]:
# Fin